## Imports

In [ ]:
# Data analysis packages:
import pandas as pd
import numpy as np
pd.set_option('display.float_format', lambda x: '%.3f'%x)
#from datetime import datetime as dt

# Visualization packages:
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as patches
%matplotlib inline

from matplotlib.dates import DateFormatter
import matplotlib.dates as mdates

import warnings
import itertools
import datetime as dt
from IPython.display import HTML # to see everything
plt.style.use('seaborn-darkgrid')
warnings.filterwarnings("ignore")

Import sklearn and statsmodels 

In [ ]:
# import sklearn 
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, StandardScaler
from sklearn.metrics import mean_squared_error

# import Statsmodels 
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
from statsmodels.tools.eval_measures import rmse, aic
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.gofplots import qqplot
import statsmodels.api as sm

Import from Keras

In [ ]:
# !pip install tf-nightly-2.0-preview
import tensorflow as tf

# import Sequential
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Input, Dense, LSTM, Embedding
from tensorflow.python.keras.optimizers import RMSprop
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau
# print 
print(tf.__version__)
print(tf.keras.__version__)

Import my functions

In [ ]:
# import plots as p
# import function as f
# import data_testing as dt
# import data_prep as dp
%load_ext autoreload
%autoreload 2

# class example

In [ ]:
def plot_series(time, series, format="-", start=0, end=None):
    plt.plot(time[start:end], series[start:end], format)
    plt.xlabel("Time")
    plt.ylabel("Value")
    plt.grid(True)

def trend(time, slope=0):
    return slope * time

def seasonal_pattern(season_time):
    """Just an arbitrary pattern, you can change it if you wish"""
    return np.where(season_time < 0.4,
                    np.cos(season_time * 2 * np.pi),
                    1 / np.exp(3 * season_time))

def seasonality(time, period, amplitude=1, phase=0):
    """Repeats the same pattern at each period"""
    season_time = ((time + phase) % period) / period
    return amplitude * seasonal_pattern(season_time)

def noise(time, noise_level=1, seed=None):
    rnd = np.random.RandomState(seed)
    return rnd.randn(len(time)) * noise_level

time = np.arange(4 * 365 + 1, dtype="float32")
baseline = 10
series = trend(time, 0.1)  
baseline = 10
amplitude = 40
slope = 0.05
noise_level = 5

# Create the series
series = baseline + trend(time, slope) + seasonality(time, period=365, amplitude=amplitude)
# Update with noise
series += noise(time, noise_level, seed=42)

split_time = 1000
time_train = time[:split_time]
x_train = series[:split_time]
time_valid = time[split_time:]
x_valid = series[split_time:]

window_size = 20
batch_size = 32
shuffle_buffer_size = 1000

In [ ]:
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    dataset = tf.data.Dataset.from_tensor_slices(series)
    dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))
    dataset = dataset.shuffle(shuffle_buffer).map(lambda window: (window[:-1], window[-1]))
    dataset = dataset.batch(batch_size).prefetch(1)
    return dataset

In [ ]:
print(type(x_train))
print(x_train.shape)
print(type(x_valid))
print(x_valid.shape)

In [ ]:
windowed_dataset(x_train, window_size, batch_size, shuffle_buffer_size)

# Load Data -- Electricity

In [ ]:
office = pd.read_csv('data/office_1249.csv', index_col=['timestamp'], parse_dates=['timestamp'])
usecols =['meter', 'meter_reading', 'air_temperature', 'cloud_coverage', 'dew_temperature','precip_depth_1_hr', 'sea_level_pressure', 'wind_direction', 'wind_speed']
# office = office[office[usecols]]
office = office[usecols]
office_temp = office.ffill()
office_data = office_temp.bfill()
office_data.head(3)

In [ ]:
# electricity
df_elc = office_data[office_data['meter']==0]
df_elc.drop(columns=['meter'], inplace=True)
df_elc.head(3)

In [ ]:
df_reading = df_elc['meter_reading']
df_elc.loc['2016-07-07':'2016-07-14']['meter_reading'].plot(figsize=(15,8))
# df_reading.plot(figsize=(15,8))

In [ ]:
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    dataset = tf.data.Dataset.from_tensor_slices(series)
    dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))
    dataset = dataset.shuffle(shuffle_buffer).map(lambda window: (window[:-1], window[-1]))
    dataset = dataset.batch(batch_size).prefetch(1)
    return dataset

# Features and labels

In [ ]:
# target_names = ['meter_reading', 'air_temperature']
target_names = ['meter_reading', 'air_temperature'] # list(df_elc.columns)
shift_days = 1
shift_steps = shift_days * 24 # number of hours.
df_targets = df_elc[target_names].shift(-shift_steps)
df_targets.head(3)

In [ ]:
df_features = df_elc
df_features.iloc[24:27]

In [ ]:
# x_data
x_data = df_features.values[0:-shift_steps]
print(type(x_data))
print('Feature Shape:', x_data.shape)
 
# y_data
y_data = df_targets.values[:-shift_steps]
print(type(y_data))
print('Target Shape:', y_data.shape)

In [ ]:
# These are the input-signals for the training- and test-sets:
num_data = len(x_data)
train_split = 0.9
num_train = int(train_split * num_data)
num_test = num_data - num_train
print('Training observations:', num_train)
print('Validation observations:', num_test)
x_train = x_data[0:num_train]
x_test  = x_data[num_train:]
print('Total observations:', (len(x_train) + len(x_test)))

In [ ]:
x_train.shape

In [ ]:
x_train

In [ ]:
# These are the output-signals for the training- and test-sets:
y_train = y_data[0:num_train]
y_test = y_data[num_train:]
print('Total test observations:', (len(y_train) + len(y_test)))

In [ ]:
y_train.shape

In [ ]:
num_x_signals = x_data.shape[1]
num_y_signals = y_data.shape[1]
print('Number of input-signals:', num_x_signals, 'and shape:', x_train.shape )
print('Number of output-signals:', num_y_signals, 'and shape', y_train.shape)

## Scaled Data

In [ ]:
x_data[24:27] # I shifted the data by one day

In [ ]:
print('Min and Max x_train data')
print('    Min:', np.min(x_train))
print('    Max:', np.max(x_train))
x_scaler = MinMaxScaler()
x_train_scaled = x_scaler.fit_transform(x_train)
print('Min and Max x_train_scaled data')
print('    Min:', np.min(x_train_scaled))
print('    Max:', np.max(x_train_scaled))

In [ ]:
x_test[24:27]

In [ ]:
y_test

In [ ]:
x_test_scaled = x_scaler.transform(x_test)
print('Min and Max x_test data')
print('    Min:', np.min(x_test))
print('    Max:', np.max(x_test))
y_scaler = MinMaxScaler()
y_train_scaled = y_scaler.fit_transform(y_train)
y_test_scaled = y_scaler.transform(y_test)
print('Min and Max y_test_scaled data')
print('    Min:', np.min(y_test_scaled))
print('    Max:', np.max(y_test_scaled))

In [ ]:
print(x_train_scaled.shape)
print(y_train_scaled.shape)
print(x_test_scaled.shape)
print(y_test_scaled.shape)
y_train_scaled

## Data Generator

In [ ]:
print(x_train_scaled.shape)
print(y_train_scaled.shape)

In [ ]:
def batch_generator(batch_size, sequence_length):
    """
    Generator function for creating random batches of training-data.
    """

    # Infinite loop.
    while True:
        # Allocate a new array for the batch of input-signals.
        x_shape = (batch_size, sequence_length, num_x_signals)
        x_batch = np.zeros(shape=x_shape, dtype=np.float16)

        # Allocate a new array for the batch of output-signals.
        y_shape = (batch_size, sequence_length, num_y_signals)
        y_batch = np.zeros(shape=y_shape, dtype=np.float16)

        # Fill the batch with random sequences of data.
        for i in range(batch_size):
            # Get a random start-index.
            # This points somewhere into the training-data.
            idx = np.random.randint(num_train - sequence_length)
            
            # Copy the sequences of data starting at this index.
            x_batch[i] = x_train_scaled[idx:idx+sequence_length]
            y_batch[i] = y_train_scaled[idx:idx+sequence_length]
        
        yield (x_batch, y_batch)

```python
# reshape input to be [samples, time steps, features]
```

In [ ]:
# reshape input to be [samples, time steps, features]
batch_size = 72
sequence_length = 24

# create the batch-generator
generator = batch_generator(batch_size, sequence_length)

# test the batch-generator to see if it works 
x_batch, y_batch = next(generator)
print(x_batch.shape)
print(y_batch.shape)

we get a batch output where we have a batch size of xxx sequences, each sequence has yyy, and zzz input signals and zzz' output signals. 

In [ ]:
batch = 0   # First sequence in the batch.
signal = 0  # First signal from the 20 input-signals.
seq = x_batch[batch, :, signal]
plt.plot(seq);

In [ ]:
seq = y_batch[batch, :, signal]
plt.plot(seq);

In [ ]:
validation_data = (np.expand_dims(x_test_scaled, axis=0),
                   np.expand_dims(y_test_scaled, axis=0))

## Create the Recurrent Neural Network

In [ ]:
# from tf.keras.models import Sequential  # This does not work!
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Input, Dense, GRU, Embedding
from tensorflow.python.keras.optimizers import RMSprop
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau

In [ ]:

print(x_batch.shape)
print(y_batch.shape)

In [ ]:
# model Architecture
model = Sequential()
model.add(GRU(units=512,
              return_sequences=True,
              input_shape=(None, num_x_signals,)))

model.add(Dense(num_y_signals, activation='sigmoid'))

model.summary()

#### -- start

In [ ]:
# model Architecture
model_s = Sequential()
model_s.add(LSTM(units=4,
              return_sequences=True,
#               input_shape=(None, num_x_signals,)
                 input_shape=(None, num_x_signals,)
                ))

model_s.add(Dense(num_y_signals, activation='sigmoid'))

model_s.summary()

In [ ]:
# from the other code
model_lstm = Sequential()
model_lstm.add(LSTM(units=100,
               return_sequences=True,
               input_shape=(x_batch.shape[1], x_batch.shape[2])))
model_lstm.add(Dropout(0.2))
model_lstm.add(Dense(num_y_signals))
model_lstm.compile(loss='mean_squared_error', optimizer='adam')

history = model.fit(x_batch, y_batch, epochs=2, 
                    batch_size=70, validation_data=(X_test, Y_test), 
                    callbacks=[EarlyStopping(monitor='val_loss', patience=10)], verbose=1, shuffle=False)

# # Training Phase

model.fit_generator(generator=generator,
                    epochs=2,
                    steps_per_epoch=100,
                    validation_data=validation_data,
                    callbacks=callbacks)

In [ ]:
# model = Sequential()
# model.add(LSTM(100, input_shape=(X_train.shape[1], X_train.shape[2])))
# model.add(Dropout(0.2))
# model.add(Dense(1))
# model.compile(loss='mean_squared_error', optimizer='adam')

# history = model.fit(X_train, Y_train, epochs=20, batch_size=70, validation_data=(X_test, Y_test), 
#                     callbacks=[EarlyStopping(monitor='val_loss', patience=10)], verbose=1, shuffle=False)

# # Training Phase
# model.summary()

#### - end

In [ ]:
if False:
    from tensorflow.python.keras.initializers import RandomUniform

    # Maybe use lower init-ranges.
    init = RandomUniform(minval=-0.05, maxval=0.05)

    model.add(Dense(num_y_signals,
                    activation='linear',
                    kernel_initializer=init))


## Loss Function

In [ ]:
warmup_steps = 50

In [ ]:
def loss_mse_warmup(y_true, y_pred):
    """
    Calculate the Mean Squared Error between y_true and y_pred,
    but ignore the beginning "warmup" part of the sequences.
    
    y_true is the desired output.
    y_pred is the model's output.
    """

    # The shape of both input tensors are:
    # [batch_size, sequence_length, num_y_signals].

    # Ignore the "warmup" parts of the sequences
    # by taking slices of the tensors.
    y_true_slice = y_true[:, warmup_steps:, :]
    y_pred_slice = y_pred[:, warmup_steps:, :]

    # These sliced tensors both have this shape:
    # [batch_size, sequence_length - warmup_steps, num_y_signals]

    # Calculate the MSE loss for each value in these tensors.
    # This outputs a 3-rank tensor of the same shape.
    loss = tf.losses.mean_squared_error(labels=y_true_slice,
                                        predictions=y_pred_slice)

    # Keras may reduce this across the first axis (the batch)
    # but the semantics are unclear, so to be sure we use
    # the loss across the entire tensor, we reduce it to a
    # single scalar with the mean function.
    loss_mean = tf.reduce_mean(loss)

    return loss_mean

## Compile Model

In [ ]:
# optimization  
optimizer = RMSprop(lr=1e-3)
model.compile(loss=loss_mse_warmup, optimizer=optimizer)

In [ ]:
model.summary()

the output is a tensor with an arbitrary batch size and arbitrary sequence length and two output signals.

## Callback Functions

In [ ]:
path_checkpoint = '23_checkpoint.keras'
callback_checkpoint = ModelCheckpoint(filepath=path_checkpoint,
                                     monitor='val_loss',
                                      verbose=1,
                                      save_weights_only=True,
                                      save_best_only=True)
callback_early_stopping = EarlyStopping(monitor='val_loss',
                                        patience=5, verbose=1)

In [ ]:
callback_tensorboard = TensorBoard(log_dir='./23_logs/',
                                   histogram_freq=0,
                                   write_graph=False)

In [ ]:
callback_reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                                       factor=0.1,
                                       min_lr=1e-4,
                                       patience=0,
                                       verbose=1)

In [ ]:
callbacks = [callback_early_stopping,
             callback_checkpoint,
             callback_tensorboard,
             callback_reduce_lr]

## Train  the Recurrent Neural Network

In [ ]:
%%time
model.fit_generator(generator=generator,
                    epochs=2,
                    steps_per_epoch=100,
                    validation_data=validation_data,
                    callbacks=callbacks)

In [ ]:
result = model.evaluate(x=np.expand_dims(x_test_scaled, axis=0),
                        y=np.expand_dims(y_test_scaled, axis=0))

In [ ]:
print("loss (test-set):", result)

In [ ]:
def plot_comparison(start_idx, length=100, train=True):
    """
    Plot the predicted and true output-signals.
    
    :param start_idx: Start-index for the time-series.
    :param length: Sequence-length to process and plot.
    :param train: Boolean whether to use training- or test-set.
    """
    
    if train:
        # Use training-data.
        x = x_train_scaled
        y_true = y_train
    else:
        # Use test-data.
        x = x_test_scaled
        y_true = y_test
    
    # End-index for the sequences.
    end_idx = start_idx + length
    
    # Select the sequences from the given start-index and
    # of the given length.
    x = x[start_idx:end_idx]
    y_true = y_true[start_idx:end_idx]
    
    # Input-signals for the model.
    x = np.expand_dims(x, axis=0)

    # Use the model to predict the output-signals.
    y_pred = model.predict(x)
    
    # The output of the model is between 0 and 1.
    # Do an inverse map to get it back to the scale
    # of the original data-set.
    y_pred_rescaled = y_scaler.inverse_transform(y_pred[0])
    
    # For each output-signal.
    for signal in range(len(target_names)):
        # Get the output-signal predicted by the model.
        signal_pred = y_pred_rescaled[:, signal]
        
        # Get the true output-signal from the data-set.
        signal_true = y_true[:, signal]

        # Make the plotting-canvas bigger.
        plt.figure(figsize=(15,5))
        
        # Plot and compare the two signals.
        plt.plot(signal_true, label='true')
        plt.plot(signal_pred, label='pred')
        
        # Plot grey box for warmup-period.
        p = plt.axvspan(0, warmup_steps, facecolor='black', alpha=0.15)
        
        # Plot labels etc.
        plt.ylabel(target_names[signal])
        plt.legend()
        plt.show()

In [ ]:
plot_comparison(start_idx=4000, length=1000, train=True)